In [ ]:
pip install scikit-surprise


  Using cached scikit-surprise-1.1.3.tar.gz (771 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162674 sha256=822cae9d3a612a3250742ea1eedf65ddd75b1d4571144eac362b50a79dd0472f
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


Шаг 1: Импорт необходимых библиотек
Для начала импортируем необходимые библиотеки, включая Surprise для создания и обучения модели рекомендательной системы.

In [ ]:
import random
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

from collections import defaultdict
from surprise import Dataset, SVD

Шаг 2: Создание собственного датасета
Давайте создадим собственный датасет, который будет содержать информацию о пользователях, товарах и их оценках. В этом примере мы создадим случайные данные для пользователей и товаров.

In [ ]:
# Создаем список пользователей и инструментов
users = [str(i) for i in range(1, 101)]
tools = [str(i) for i in range(1, 201)]

# Генерируем случайные оценки пользователей для инструментов
data = []
for user in users:
    for tool in tools:
        rating = random.randint(1, 5)
        data.append([user, tool, rating])

# Создаем DataFrame из данных
df = pd.DataFrame(data, columns=['user_id', 'tool_id', 'rating'])

# Создаем объект Reader для определения формата данных
reader = Reader(rating_scale=(1, 5))

# Создаем датасет из DataFrame и объекта Reader
dataset = Dataset.load_from_df(df[['user_id', 'tool_id', 'rating']], reader)

Шаг 3: Разделение данных на обучающий и тестовый наборы
Для оценки производительности модели разделим данные на обучающий и тестовый наборы.

In [ ]:
trainset, testset = train_test_split(dataset, test_size=0.2, random_state=42)


Шаг 4: Создание и обучение модели
Теперь создадим модель рекомендательной системы. В этом примере мы используем модель SVD.

In [ ]:
model = SVD()
model.fit(trainset)

Шаг 5: Получение прогнозов и оценка производительности модели
Мы получим прогнозы на тестовом наборе и оценим производительность модели.

In [ ]:
predictions = model.test(testset)

mae = accuracy.mae(predictions)
rmse = accuracy.rmse(predictions)

print(f'MAE: {mae}')
print(f'RMSE: {rmse}')

MAE:  1.2764
RMSE: 1.4901
MAE: 1.276361705248178
RMSE: 1.4901490723653616


Шаг 6: Получение рекомендаций для конкретного пользователя
Теперь мы можем получить персонализированные рекомендации для конкретного пользователя. Для этого выберем случайного пользователя и получим рекомендации.

In [ ]:
# Выбираем случайного пользователя
user_id = random.choice(users)

# Получаем топ N рекомендаций для пользователя
top_n = get_top_n(predictions, n=10)

print(f"Рекомендации инструментов для пользователя {user_id}:")
for tool_id, rating in top_n[user_id]:
    print(f"Инструмент ID: {tool_id}, Рейтинг: {rating}")


Рекомендации инструментов для пользователя 41:
Инструмент ID: 176, Рейтинг: 3.927402354803331
Инструмент ID: 93, Рейтинг: 3.8504482012720964
Инструмент ID: 42, Рейтинг: 3.8431308388063705
Инструмент ID: 97, Рейтинг: 3.6202412124996144
Инструмент ID: 3, Рейтинг: 3.5534935685618363
Инструмент ID: 17, Рейтинг: 3.4631409384121725
Инструмент ID: 27, Рейтинг: 3.446649325542008
Инструмент ID: 200, Рейтинг: 3.430779425417518
Инструмент ID: 71, Рейтинг: 3.3918678719428614
Инструмент ID: 18, Рейтинг: 3.3876695046020746


Шаг 7: Настройка параметров модели (по желанию)
Для настройки параметров модели, мы можем использовать методы подбора гиперпараметров, такие как GridSearchCV из scikit-learn.

In [ ]:
# Создаем объект модели SVD
model = SVD()

# Используем GridSearchCV для подбора параметров
grid_search = GridSearchCV(SVD,
                           param_grid=param_grid,
                           measures=['rmse'],
                           cv=5)

# Выполняем GridSearchCV
grid_search.fit(dataset)

# Получаем наилучшие параметры
best_params = grid_search.best_params['rmse']
print(f'Наилучшие параметры: {best_params}')

# Создаем объект модели SVD с наилучшими параметрами
model = SVD(n_factors=best_params['n_factors'],
            n_epochs=best_params['n_epochs'],
            lr_all=best_params['lr_all'],
            reg_all=best_params['reg_all'])

# Обучаем модель с наилучшими параметрами
model.fit(trainset)

# Делаем предсказания на тестовом наборе
predictions = model.test(testset)

# Вычисляем показатели производительности
mae = accuracy.mae(predictions)
rmse = accuracy.rmse(predictions)

print(f'MAE: {mae}')
print(f'RMSE: {rmse}')


Наилучшие параметры: {'n_factors': 200, 'n_epochs': 20, 'lr_all': 0.002, 'reg_all': 0.2}
MAE:  1.2260
RMSE: 1.4246
MAE: 1.2260246412354827
RMSE: 1.4246174898717248


Шаг 8: Интеграция в магазин
Чтобы интегрировать рекомендательную систему в магазин, вы можете создать функцию, которая будет предоставлять рекомендации на основе предпочтений пользователя. Это может быть вызвано на странице пользователя или странице инструмента.

In [ ]:
def get_top_n(predictions, user_id, n=10):
    """Получение топ-N рекомендаций для пользователя"""
    top_n = defaultdict(list)
    for uid, iid, _, est, _ in predictions:
        if uid == user_id:
            top_n[uid].append((iid, est))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n[user_id]

# Использование функции в магазине
user_id = '42'
recommended_toolss = get_top_n(predictions, user_id)
